# 02 - Transformação dos Dados da Bronze (Silver Layer)

1. Carregar os dados extraídos da bronze layer (`.json`)
2. Limpar os dados e selecionar apenas colunas úteis
3. Salvar os dados em formato `.parquet` particionados por `state` (estado)


In [1]:
import pandas as pd
import os
import glob 
import re
from datetime import datetime
import shutil

In [2]:
bronze_files = glob.glob("../notebooks/data/bronze/*.json")
print(bronze_files)

['../notebooks/data/bronze\\breweries_2025-06-17.json', '../notebooks/data/bronze\\breweries_2025-06-18.json', '../notebooks/data/bronze\\breweries_2025-06-19.json']


In [3]:
bronze_files = glob.glob("../notebooks/data/bronze/*.json")

In [4]:
def extrair_data(nome_arquivo):
    match = re.search(r"(\d{4}-\d{2}-\d{2})", nome_arquivo)
    if match:
        return datetime.strptime(match.group(1), "%Y-%m-%d")
    return datetime.min

In [5]:
bronze_files.sort(key=extrair_data, reverse=True)

In [6]:
if bronze_files:
    arquivo_mais_recente = bronze_files[0]
    print(f"Arquivo mais recente encontrado: {arquivo_mais_recente}")

    df = pd.read_json(arquivo_mais_recente)
    print("------------------------------------------------")
    print(df.head(0))
    print("------------------------------------------------")
else:
    print("------------------------------------------------")
    print("Nenhum arquivo JSON encontrado na camada bronze")
    print("------------------------------------------------")

Arquivo mais recente encontrado: ../notebooks/data/bronze\breweries_2025-06-19.json
------------------------------------------------
Empty DataFrame
Columns: [id, name, brewery_type, address_1, address_2, address_3, city, state_province, postal_code, country, longitude, latitude, phone, website_url, state, street]
Index: []
------------------------------------------------


In [7]:
# Carregar o JSON para um DataFrame

df = pd.read_json(arquivo_mais_recente)
print(f"Total de registros: {len(df)}")
df.head(1)


Total de registros: 50


,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,1716 Topeka St,None,NaN,Norman,Oklahoma,73069-8224,United States,-97.468182,35.257389,4.058160e+09,http://www.405brewing.com,Oklahoma,1716 Topeka St


In [8]:
#Entendendo os tipos de dados 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              50 non-null     object 
 1   name            50 non-null     object 
 2   brewery_type    50 non-null     object 
 3   address_1       47 non-null     object 
 4   address_2       1 non-null      object 
 5   address_3       0 non-null      float64
 6   city            50 non-null     object 
 7   state_province  50 non-null     object 
 8   postal_code     50 non-null     object 
 9   country         50 non-null     object 
 10  longitude       40 non-null     float64
 11  latitude        40 non-null     float64
 12  phone           45 non-null     float64
 13  website_url     39 non-null     object 
 14  state           50 non-null     object 
 15  street          47 non-null     object 
dtypes: float64(4), object(12)
memory usage: 6.4+ KB


In [9]:
# 1. Selecionar apenas colunas úteis
colunas_uteis = [
    'id', 'name', 'brewery_type', 'city', 'state_province',
    'postal_code', 'country', 'longitude', 'latitude', 'phone', 'website_url'
]
df = df[colunas_uteis]

In [10]:
df = df.rename(columns = {
    
    'id':'ID',
    'name':'NAME',
    'brewery_type':'TYPE BREWERY',
    'city':'CITY',
    'state_province':'STATE PROVINCE',
    'country':'COUNTRY',
    'longitude':'LONGITUDE',
    'latitude':'LATITUDE',
    'phone':'PHONE',
    'website_url':'URL WEBSITE',
    'postal_code':'POSTAL CODE'
})
df.head()

,ID,NAME,TYPE BREWERY,CITY,STATE PROVINCE,POSTAL CODE,COUNTRY,LONGITUDE,LATITUDE,PHONE,URL WEBSITE
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,Norman,Oklahoma,73069-8224,United States,-97.468182,35.257389,4.058160e+09,http://www.405brewing.com
1,9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,micro,Austin,Texas,78745-1197,United States,NaN,NaN,5.129212e+09,http://www.512brewing.com
2,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,1 of Us Brewing Company,micro,Mount Pleasant,Wisconsin,53406-3920,United States,-87.883364,42.720108,2.624848e+09,https://www.1ofusbrewing.com
3,6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,large,Bend,Oregon,97701-9847,United States,-121.281706,44.086835,5.415851e+09,http://www.10barrel.com
4,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,10 Barrel Brewing Co,large,Bend,Oregon,97703-2465,United States,-121.328802,44.057565,5.415851e+09,None


In [11]:
df['TYPE BREWERY'] = df['TYPE BREWERY'].str.strip().str.upper()
df['STATE PROVINCE'] = df['STATE PROVINCE'].str.strip().str.upper()
df['COUNTRY'] = df['COUNTRY'].str.strip().str.upper()
df['CITY'] = df['CITY'].str.strip().str.upper()

In [12]:
df['PHONE'] = df['PHONE'].astype('Int64')

In [13]:
df['URL WEBSITE'] = df['URL WEBSITE'].str.replace(r'^https?://', '', regex=True)
df.head()

,ID,NAME,TYPE BREWERY,CITY,STATE PROVINCE,POSTAL CODE,COUNTRY,LONGITUDE,LATITUDE,PHONE,URL WEBSITE
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,MICRO,NORMAN,OKLAHOMA,73069-8224,UNITED STATES,-97.468182,35.257389,4058160490,www.405brewing.com
1,9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,MICRO,AUSTIN,TEXAS,78745-1197,UNITED STATES,NaN,NaN,5129211545,www.512brewing.com
2,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,1 of Us Brewing Company,MICRO,MOUNT PLEASANT,WISCONSIN,53406-3920,UNITED STATES,-87.883364,42.720108,2624847553,www.1ofusbrewing.com
3,6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,LARGE,BEND,OREGON,97701-9847,UNITED STATES,-121.281706,44.086835,5415851007,www.10barrel.com
4,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,10 Barrel Brewing Co,LARGE,BEND,OREGON,97703-2465,UNITED STATES,-121.328802,44.057565,5415851007,None


In [14]:
df = df[(df['LATITUDE'].between(-90, 90)) & (df['LONGITUDE'].between(-180, 180))]
df.head()

,ID,NAME,TYPE BREWERY,CITY,STATE PROVINCE,POSTAL CODE,COUNTRY,LONGITUDE,LATITUDE,PHONE,URL WEBSITE
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,MICRO,NORMAN,OKLAHOMA,73069-8224,UNITED STATES,-97.468182,35.257389,4058160490,www.405brewing.com
2,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,1 of Us Brewing Company,MICRO,MOUNT PLEASANT,WISCONSIN,53406-3920,UNITED STATES,-87.883364,42.720108,2624847553,www.1ofusbrewing.com
3,6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,LARGE,BEND,OREGON,97701-9847,UNITED STATES,-121.281706,44.086835,5415851007,www.10barrel.com
4,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,10 Barrel Brewing Co,LARGE,BEND,OREGON,97703-2465,UNITED STATES,-121.328802,44.057565,5415851007,None
5,e432899b-7f58-455f-9c7b-9a6e2130a1e0,10 Barrel Brewing Co,LARGE,PORTLAND,OREGON,97209-2620,UNITED STATES,-122.685506,45.525979,5032241700,www.10barrel.com


In [15]:
df = df.fillna({
                'ID':'Unknown',
                'NAME':'Unknown',
                'TYPE BREWERY':'Unknown',
                'CITY':'Unknown',
                'STATE PROVINCE':'Unknown',
                'POSTAL CODE':'Unknown',
                'COUNTRY':'Unknown',
                'LONGITUDE': 0,
                'LATITUDE': 0,
                'PHONE':0,
                'URL WEBSITE':'Unknown'
               }
                
              )
#df.head()

In [16]:
silver_path = "../notebooks/data/silver/breweries.parquet"

In [17]:
if os.path.exists(silver_path):
    files = glob.glob(os.path.join(silver_path, "*"))
    for f in files:
        if os.path.isdir(f):
            shutil.rmtree(f)
        else:
            os.remove(f)

In [18]:
df.to_parquet("../notebooks/data/silver/breweries.parquet", partition_cols=['STATE PROVINCE'], index=False)